# Trains and Tests PLDA

In [3]:
#!/usr/bin/env python3.7
import os
import sys
sys.path.insert(0, "/export/c10/lmorove1/PythonLibs/plda")
sys.path.insert(0, "/home/sjoshi/codes/python/BeatPD/code")
import plda
import numpy as np
import matplotlib.pyplot as plt
import kaldi_io
import argparse
import pickle
from sklearn.decomposition import PCA

import pandas as pd 

from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error

#PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Confusion matrix
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [4]:
DATA_PATH = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/'

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

/home/sjoshi/codes/python/BeatPD/code
accelerometer_plots	    Pipeline.ipynb
analyze_data_cleaned.ipynb  pretrain_sjoshi.ipynb
analyze_data.py		    pretrain_sjoshi-remove-data.ipynb
bobivectors.ipynb	    pretrain_sjoshi-remove-data-twoclass.ipynb
bobivectors.py		    pretrain_sjoshi-remove-data-twoclass-onlyAccl.ipynb
Graphs.ipynb		    __pycache__
LSTM_BeatPD.ipynb	    README.md
models			    read_pkl_files.ipynb
out_train_ivectors	    requirements.txt
pca_knn_bpd.py		    train_ivectors.sh
PCA_KNN.ipynb		    tsfresh.ipynb
pca_plda_bpd.py		    utils
/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 data	mfcc		   pca_pldapy.ipynb  runFor.sh~  sid
cmd.sh~  exp	newpldatest.ipynb  README.txt	     run.sh	 steps
conf	 local	path.sh		   runFor.sh	     run.sh~	 utils


In [5]:
def pca(sFileTrai, sFileTest, iComponents):
# os.chdir("/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k")
# sFileTrai="/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k/exp/sre18nn/xvectors_Training_Fold1/xvector.scp"
# sOut = Folder where to store results 
# iComponents = nb of classes (nb of measurements_id? or nb of subject_ids?)
    dIvecTrai = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTrai) }
    vTrai= pd.DataFrame((list(dIvecTrai.values())))
    vLTrai = np.array([x[-1] for x in np.array(list(dIvecTrai.keys()))])
    
    # FIXME : For realPD, we need more than -5 (CIS-PD subject_id is 4 characters long)
    # FIXME REAL-PD it's not only int 
    vTraiSubjectId = np.array(([int(x[-5:-1]) for x in np.array(list(dIvecTrai.keys()))]))

    dIvecTest = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTest) }
    vTest=np.array(list(dIvecTest.values()), dtype=float)
    vLTest=np.array([x[-1] for x in np.array(list(dIvecTest.keys()))])
    vTestSubjectId = np.array([int(x[-5:-1]) for x in np.array(list(dIvecTest.keys()))])

    print('before transform : ', vTrai.shape)
    print('before transform : ', vTest.shape)

    if isinstance(iComponents, str):
            iComponents=int(iComponents)

    pca = PCA(n_components=iComponents, svd_solver='randomized', whiten=True)
    pca.fit(vTrai)


    vTraiPCA=pca.transform(vTrai)
    vTestPCA=pca.transform(vTest)

    print('after transform : ', vTraiPCA.shape)
    print('after transform : ', vTestPCA.shape)
    return vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId

In [6]:
sFileTrai = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Training_Fold0/ivector.scp'
sFileTest = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Testing_Fold0/ivector.scp'
iComponents = 50 


In [7]:
vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId = pca(sFileTrai, sFileTest, iComponents)


before transform :  (1386, 300)
before transform :  (381, 300)
after transform :  (1386, 50)
after transform :  (381, 50)


In [8]:
# # To compute the final score as per the challenge
# mse_training_per_subjectid=[]
# mse_test_per_subjectid=[]
# train_nb_files_per_subjectid=[]
# test_nb_files_per_subjectid=[]

# # To compute the mean accuracy
# glob_trai_pred=[]
# glob_test_pred=[]
# glob_trai_true=[]
# glob_test_true=[]


# vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId = pca(sFileTrai, sFileTest, iComponents)

# for subject_id in np.unique(vTraiSubjectId):
#     print('----- ' + str(subject_id) + '----- ')
#     knn = KNeighborsClassifier(n_neighbors=iNeighbors)

#     # Filter vTraiPCA and vLTraiPCA for one subject_id
#     indices_subject_id = np.where(vTraiSubjectId == subject_id) # HAPPY
#     vTraiPCA_subjectid = vTraiPCA[indices_subject_id]
#     vLTrai_subjectid = vLTrai[indices_subject_id]

#     # Filter vTestPCA and vLTestPCA for one subject_id
#     indices_subject_id = np.where(vTestSubjectId == subject_id)
#     vTestPCA_subjectid = vTestPCA[indices_subject_id]
#     vLTest_subjectid = vLTest[indices_subject_id]

#     knn.fit(vTraiPCA_subjectid, vLTrai_subjectid)

#     print('Training accuracy: ', knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
#     print('Testing accuracy: ', knn.score(vTestPCA_subjectid, vLTest_subjectid))

#     # Predicting on the training and test data
#     predictionsTrai = knn.predict(vTraiPCA_subjectid)
#     predictions = knn.predict(vTestPCA_subjectid)

#     # Converting all strings to int for MSE
#     vLTrai_subjectid = [int(i) for i in vLTrai_subjectid]
#     predictionsTrai = [int(i) for i in predictionsTrai]
#     #vLTest_subjectid = [int(i) for i in vLTest_subjectid]
#     predictions = [int(i) for i in predictions]

#     # Computing the accuracy
#     glob_trai_pred=np.append(glob_trai_pred,predictionsTrai,axis=0)
#     glob_test_pred=np.append(glob_test_pred,predictions,axis=0)
#     glob_trai_true=np.append(glob_trai_true,vLTrai_subjectid,axis=0)
#     glob_test_true=np.append(glob_test_true,vLTest_subjectid,axis=0)

#     # Building a list of the MSEk
#     mse_training_per_subjectid = np.append(mse_training_per_subjectid,
#                                            (mean_squared_error(vLTrai_subjectid, predictionsTrai) /  len(vLTrai_subjectid)))
#     mse_test_per_subjectid = np.append(mse_test_per_subjectid,
#                                         (mean_squared_error(vLTest_subjectid, predictions) / len(vLTest_subjectid)))
#     train_nb_files_per_subjectid.append(len(vLTrai_subjectid))
#     test_nb_files_per_subjectid.append(len(vLTest_subjectid))

# print(mse_training_per_subjectid)
# print('Global training accuracy: {}'.format((glob_trai_true == glob_trai_pred).mean()))
# print('Global testing accuracy: {}'.format((glob_test_true == glob_test_pred).mean()))
# print('PCAComponents: {}'.format((iComponents)))
# print('iNeighbors: {}'.format((iNeighbors)))
# # print('True Labels Training:')
# # print(glob_trai_true)
# # print('#')
# # print('#')
# # print('Predicted Labels Training:')
# # print(glob_trai_pred)
# # print('#')
# # print('#')
# # print('True Labels Testing:')
# # print(glob_test_true)
# # print('#')
# # print('#')
# # print('Predicted Labels Testing:')
# # print(glob_test_pred)
# # print('#')
# # print('#')

# if not  os.path.isdir(sOut):
#         os.mkdir(sOut)
# sObjname='objs_'+str(iComponents)+'k_'+str(iNeighbors)'.pkl'
# with open(os.path.join(sOut,sObjname), 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump([glob_trai_pred,glob_trai_true,glob_test_pred,glob_test_true, \
#                 mse_training_per_subjectid,mse_test_per_subjectid, \
#                 train_nb_files_per_subjectid,test_nb_files_per_subjectid], f)

In [114]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# To compute the final score as per the challenge
mse_training_per_subjectid=[]
mse_test_per_subjectid=[]
train_nb_files_per_subjectid=[]
test_nb_files_per_subjectid=[]

# To compute the mean accuracy
glob_trai_pred=[]
glob_test_pred=[]
glob_trai_true=[]
glob_test_true=[]
    
for subject_id in np.unique(vTraiSubjectId):

    print('----- ' + str(subject_id) + '----- ')
    knn = KNeighborsClassifier(n_neighbors=6)
    
    # Filter vTraiPCA and vLTraiPCA for one subject_id
    indices_subject_id = np.where(vTraiSubjectId == subject_id) # HAPPY
    vTraiPCA_subjectid = vTraiPCA[indices_subject_id]
    vLTrai_subjectid = vLTrai[indices_subject_id]
    
    # Filter vTestPCA and vLTestPCA for one subject_id
    indices_subject_id = np.where(vTestSubjectId == subject_id)
    vTestPCA_subjectid = vTestPCA[indices_subject_id]
    vLTest_subjectid = vLTest[indices_subject_id]

    knn.fit(vTraiPCA_subjectid, vLTrai_subjectid)
    
#     y_labels = knn.predict(vTraiPCA_subjectid)
#     display(y_labels)

    print('Training accuracy: ', knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
    print('Testing accuracy: ', knn.score(vTestPCA_subjectid, vLTest_subjectid))
    
    # Predicting on the training and test data 
    predictionsTrai = knn.predict(vTraiPCA_subjectid)
    predictions = knn.predict(vTestPCA_subjectid)
    
    # Converting all strings to int for MSE 
    vLTrai_subjectid = [int(i) for i in vLTrai_subjectid]
    predictionsTrai = [int(i) for i in predictionsTrai]
    vLTest_subjectid = [int(i) for i in vLTest_subjectid]
    predictions = [int(i) for i in predictions]
 
    # Computing the accuracy 
    glob_trai_pred=np.append(glob_trai_pred,predictionsTrai,axis=0)
    glob_test_pred=np.append(glob_test_pred,predictions,axis=0)
    glob_trai_true=np.append(glob_trai_true,vLTrai_subjectid,axis=0)
    glob_test_true=np.append(glob_test_true,vLTest_subjectid,axis=0)
    
    a = [a - b for a, b in zip(vLTrai_subjectid, predictionsTrai)]
    print('abs substract : ', np.abs(a))
    
    # Building a list of the MSEk 
    mse_training_per_subjectid = np.append(mse_training_per_subjectid,
                                           (mean_squared_error(vLTrai_subjectid, predictionsTrai) /  len(vLTrai_subjectid)))
    mse_test_per_subjectid = np.append(mse_test_per_subjectid,
                                        (mean_squared_error(vLTest_subjectid, predictions) / len(vLTest_subjectid)))
    train_nb_files_per_subjectid.append(len(vLTrai_subjectid))
    test_nb_files_per_subjectid.append(len(vLTest_subjectid))

#     do_confusion_matrix(vLTest_subjectid, predictions)

print(type(glob_trai_true))
print('Global training accuracy: {}'.format((glob_trai_true == glob_trai_pred).mean()))
print('Global testing accuracy: {}'.format((glob_test_true == glob_test_pred).mean()))

----- 1004----- 
Training accuracy:  0.4126984126984127
Testing accuracy:  0.3157894736842105
abs substract :  [1 0 0 1 0 1 2 0 2 1 3 0 0 2 2 0 0 2 0 1 0 0 1 0 3 0 1 0 1 0 1 1 2 2 1 2 0
 3 0 1 0 0 1 0 1 0 0 0 1 2 1 0 0 2 0 1 1 2 2 2 1 3 2]
----- 1006----- 
Training accuracy:  0.8214285714285714
Testing accuracy:  0.7777777777777778
abs substract :  [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0]
----- 1007----- 
Training accuracy:  0.5525114155251142
Testing accuracy:  0.543859649122807
abs substract :  [4 0 2 0 2 0 1 2 1 0 0 0 2 0 2 0 0 0 4 0 0 1 1 0 0 3 1 0 0 1 1 4 0 0 0 0 1
 1 0 1 0 0 0 0 0 0 4 0 0 0 0 0 2 2 1 1 1 0 0 1 3 0 1 0 0 3 1 0 2 0 0 1 0 0
 2 0 2 0 1 1 0 3 1 0 0 4 0 0 0 0 0 0 0 0 0 0 0 2 0 0 1 0 0 0 1 3 0 0 1 3 0
 0 0 0 2 1 4 1 0 4 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 2 0 1 0 0 0 2 0 0 0 0 4 4
 0 0 0 1 0 1 2 0 0 0 0 4 1 0 0 0 3 4 1 0 3 0 2 0 0 0 1 3 0 2 4 1 2 0 1 0 0
 2 1 1 1 2 1 0 0 1 1 0 1 3 0 4 0 0 0 0 1 2 0 0 3 0 0 0 0 0 1 0 4 1 3]
----- 1019----- 
Training accuracy: 

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
from pca_knn_bpd import pca_knn_bpd


In [68]:
from pca_plda_bpd import pca_plda_bpd

In [112]:
pca_knn_bpd(sFileTrai, sFileTest, '/home/sjoshi/codes/python/BeatPD/code/', iComponents, 3)

----- 1004----- 
Training accuracy:  0.4603174603174603
Testing accuracy:  0.3157894736842105
----- 1006----- 
Training accuracy:  0.75
Testing accuracy:  0.6666666666666666
----- 1007----- 
Training accuracy:  0.5707762557077626
Testing accuracy:  0.543859649122807
----- 1019----- 
Training accuracy:  0.5142857142857142
Testing accuracy:  0.5
----- 1020----- 
Training accuracy:  0.8181818181818182
Testing accuracy:  0.7804878048780488
----- 1023----- 
Training accuracy:  0.47560975609756095
Testing accuracy:  0.25
----- 1032----- 
Training accuracy:  0.49640287769784175
Testing accuracy:  0.23684210526315788
----- 1034----- 
Training accuracy:  0.4666666666666667
Testing accuracy:  0.4
----- 1038----- 
Training accuracy:  0.7073170731707317
Testing accuracy:  0.6744186046511628
----- 1039----- 
Training accuracy:  0.5841584158415841
Testing accuracy:  0.41379310344827586
----- 1043----- 
Training accuracy:  0.56
Testing accuracy:  0.3333333333333333
----- 1044----- 
Training accuracy:

In [113]:
pca_plda_bpd(sFileTrai, sFileTest, '/home/sjoshi/codes/python/BeatPD/code/', iComponents)

----- 1004----- 
Previous shapes:
(63, 50)
(63,)
Shapes after removing classes:
(63, 50)
(63,)
Accuracy: 0.2631578947368421
----- 1006----- 
Previous shapes:
(28, 50)
(28,)
Shapes after removing classes:
(28, 50)
(28,)
Accuracy: 0.6666666666666666
----- 1007----- 
Previous shapes:
(219, 50)
(219,)
Shapes after removing classes:
(219, 50)
(219,)
Accuracy: 0.543859649122807
----- 1019----- 
Previous shapes:
(35, 50)
(35,)
Shapes after removing classes:
(35, 50)
(35,)
Accuracy: 0.3
----- 1020----- 
Previous shapes:
(154, 50)
(154,)
Shapes after removing classes:
(154, 50)
(154,)
Accuracy: 0.7804878048780488
----- 1023----- 
Previous shapes:
(82, 50)
(82,)
Shapes after removing classes:
(82, 50)
(82,)
Accuracy: 0.20833333333333334
----- 1032----- 
Previous shapes:
(139, 50)
(139,)
Shapes after removing classes:
(139, 50)
(139,)
Accuracy: 0.3157894736842105
----- 1034----- 
Previous shapes:
(30, 50)
(30,)
Shapes after removing classes:
(30, 50)
(30,)
Accuracy: 0.2
----- 1038----- 
Previous 

In [575]:
def final_score(mse_per_subjectid, nb_files_per_subject_id):
    numerator = np.sum([a * b for a, b in zip(np.sqrt(nb_files_per_subject_id), mse_per_subjectid)])
    denominator = np.sum(np.sqrt(nb_files_per_subject_id))
    print('numerator : ', numerator)
    print('denominator : ', denominator)
    print('Final score : ', np.divide(numerator, denominator))
    
final_score(mse_training_per_subjectid, train_nb_files_per_subjectid)
final_score(mse_test_per_subjectid, test_nb_files_per_subjectid)

In [556]:
final_score(mse_training_per_subjectid, train_nb_files_per_subjectid)
final_score(mse_test_per_subjectid, test_nb_files_per_subjectid)

numerator :  2.9956883253194277
denominator :  138.07717596772696
Final score :  0.021695753149091168
numerator :  6.065415875730853
denominator :  70.2868377625761
Final score :  0.08629518795851071


In [543]:
def do_confusion_matrix(y_test, predictions):
    print('y test : ', np.unique(y_test))
    LABELS_NEW = np.unique(y_test)
    n_classes=np.unique(y_test)
    print("Confusion Matrix:")
    confusion_matrix = metrics.confusion_matrix(y_test, predictions)
    print(confusion_matrix)
    normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

    print("")
    print("Confusion matrix (normalised to % of total test data):")
    print(normalised_confusion_matrix)

    # Plot Results:
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    plt.imshow(
        normalised_confusion_matrix,
        interpolation='nearest',
        cmap=plt.cm.rainbow
    )
    plt.title("Confusion matrix \n(normalised to % of total test data)")
    plt.colorbar()
    tick_marks = np.arange(n_classes)
    plt.xticks(tick_marks, LABELS_NEW, rotation=90)
    plt.yticks(tick_marks, LABELS_NEW)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()